In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow_core.estimator import inputs

train = pd.read_csv('exoTrain.csv')
test = pd.read_csv('exoTest.csv')

print('Dane treningowe\n', train.head())
print('Dane testowe\n', test.head())

x_train = train.drop('LABEL', axis=1)
y_train = train.LABEL-1
x_test = test.drop('LABEL', axis=1)
y_test = test.LABEL-1

Dane treningowe
    LABEL   FLUX.1   FLUX.2   FLUX.3   FLUX.4   FLUX.5   FLUX.6  FLUX.7  \
0      2    93.85    83.81    20.10   -26.98   -39.56  -124.71 -135.18   
1      2   -38.88   -33.83   -58.54   -40.09   -79.31   -72.81  -86.55   
2      2   532.64   535.92   513.73   496.92   456.45   466.00  464.50   
3      2   326.52   347.39   302.35   298.13   317.74   312.70  322.33   
4      2 -1107.21 -1112.59 -1118.95 -1095.10 -1057.55 -1034.48 -998.34   

    FLUX.8  FLUX.9  ...  FLUX.3188  FLUX.3189  FLUX.3190  FLUX.3191  \
0   -96.27  -79.89  ...     -78.07    -102.15    -102.15      25.13   
1   -85.33  -83.97  ...      -3.28     -32.21     -32.21     -24.89   
2   486.39  436.56  ...     -71.69      13.31      13.31     -29.89   
3   311.31  312.42  ...       5.71      -3.73      -3.73      30.05   
4 -1022.71 -989.57  ...    -594.37    -401.66    -401.66    -357.24   

   FLUX.3192  FLUX.3193  FLUX.3194  FLUX.3195  FLUX.3196  FLUX.3197  
0      48.57      92.54      39.32      6

In [2]:
numeric_column_headers = x_train.columns.values.tolist()
bc_fn = tf.feature_column.bucketized_column
nc_fn = tf.feature_column.numeric_column
bucketized_features = [bc_fn(source_column=nc_fn(key=column),
                            boundaries=[x_train[column].mean()])
                      for column in numeric_column_headers]

In [3]:
all_features = bucketized_features

In [4]:
batch_size=32
pi_fn = tf.compat.v1.estimator.inputs.pandas_input_fn
train_input_fn = pi_fn(x = x_train,
                      y = y_train,
                      batch_size = batch_size,
                      shuffle = True,
                      num_epochs = None)

In [5]:
eval_input_fn = pi_fn(x = x_test,
                     y = y_test,
                     batch_size = batch_size,
                     shuffle = False,
                     num_epochs = 1)

In [6]:
n_trees = 100
n_steps = 100

m_fn = tf.estimator.BoostedTreesClassifier
model = m_fn(feature_columns=all_features,
            n_trees = n_trees,
            n_batches_per_layer = batch_size,
            model_dir='./tfbtmodel')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './tfbtmodel', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
model.train(input_fn=train_input_fn, steps=n_steps)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tfbtmodel\model.ckpt-0
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input

In [8]:
results = model.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
The value of AUC returned by this may race with the update so this is deprected. Please use tf.keras.metrics.AUC instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-07-22T19:43:46Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tfbtmodel\model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 24.36841s
INFO:tensorflow:Finished evaluation at 2020-07-22-19:44:10
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.99122804, accuracy_baseline = 0.99122804, auc = 0.5000001, auc_precision_recall = 0.5043858, average_loss = 0.4602702, global_step = 100, label/mean = 0.00877193, loss = 0.46021992, precision = 0.0, prediction/mean = 0.3658339, recall = 0.0
Type is unsupported, or the types of the items

In [9]:
for key,value in sorted(results.items()):
    print('{}: {}'.format(key,value))

accuracy: 0.9912280440330505
accuracy_baseline: 0.9912280440330505
auc: 0.5000001192092896
auc_precision_recall: 0.5043858289718628
average_loss: 0.4602701961994171
global_step: 100
label/mean: 0.008771929889917374
loss: 0.4602199196815491
precision: 0.0
prediction/mean: 0.36583390831947327
recall: 0.0
